# Definições técnicas
Para possibilidar o trabalho com os modelos definidos em src, o path de busca de módulos foi alterado para incluir o caminho do projeto, de modo que o servidor jupyter consiga acessar a pasta de scripts e recuperar os modelos para manipulação do banco de dados. Assim, **é de extrema importância para o funcionamento deste notebook a correta definição do path onde o projeto será executado**.

In [21]:
base_path = "/media/bruno/Arquivos/Desenvolvimento/UBS"

import os
import sys
sys.path.append(base_path)

import pandas as pd
from dotenv import load_dotenv
from src.models.city import City
from src.database.connection import get_session

Uma das principais necessidades para seguir com uma análise padronizada é fazer a ligação entre o ID de município utilizado nas bases e o ID original da API de localidades do IBGE. O processo abaixo se encarrega de fazer este link, utilizando uma padronização de nome por meio da radicalização disponível na biblioteca NLTK e implementada no model (orm) de cidade.

In [23]:
codes = pd.read_csv(f"{base_path}/data/Codigo_Nome_Municipios.txt", sep=";", encoding = "latin1")
load_dotenv(f"{base_path}/config/.env")

session = get_session()
progress = 0
try:
    for i in range(0, len(codes)):
        token = City.tokenize(codes["NOME MUNICIPIO"].values[i])
        session.query(City).filter_by(token=token).update({
            "data_city_id": codes["CODIGO MUNICIPIO"].values[i]
        })
        progress = round((i + 1) * 100 / len(codes), 2)
        print(f"Progresso de {progress}%", end="\r")

    session.commit()

except BaseException as e:
    print(e)

finally:
    session.close()